# KnitHAT 

This notebook is step two in the process of building [KnitHAT](http://knithat.works), which recommends hat patterns from Ravelry based on user-uploaded images. KnitHAT was my Insight Data Science fellowship project, and was built in only three weeks.

This code scrapes from Ravelry information on selected patterns, based on the pattern IDs provided.

In [1]:
import time as time
import requests as rq
import pandas as pd
import re

user = [redacted]
pswd = [redacted]

/Users/ssivek/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
# Pull in hat only pattern IDs from their file; make list of IDs
mode = 'r'
ids_list = []
with open('/Users/ssivek/Insight_Projects/Ravelry_Project/Data/hat_pattern_ids.txt', mode) as f:   
    for line in f:
        ids_list.append(line.rstrip()) # add IDs in file to list

In [4]:
# print(ids_list)

In [5]:
# create list of queries to send to API and retrieve individually

all_queries = []
def create_each_query(ids_list):
    for id in ids_list:
        query_each = 'https://api.ravelry.com/patterns.json?ids=' + id
        all_queries.append(query_each) # add all individual queries to list
    return all_queries

In [8]:
len(all_queries)

77150

In [7]:
create_each_query(ids_list); # run function and hide output with semicolon

In [9]:
# create subsets of all_queries list to be able run in small batches of pattern IDs (77150 IDs total)
# tiny_batch = all_queries[:100] for testing

first = 12000 ### CHANGE THIS ONE ###
last = 16000  ### AND THIS ONE ### '''STOPPING AT 12000 FOR NOW, JUNE 11'''
sub_query = all_queries[first:last]


pattern_data = []

for query in sub_query: 
    try:
        batch = rq.get(query, auth=(user, pswd))
        data = batch.text
        pattern_data.append(data)
    except rq.exceptions.SSLError:
        time.sleep(5)
        continue        

In [10]:
df = pd.DataFrame(pattern_data)

In [11]:
df.head(5)

,0
0,"{""patterns"": {""18602"":{""comments_count"":0,""cre..."
1,"{""patterns"": {""280747"":{""comments_count"":0,""cr..."
2,"{""patterns"": {""149677"":{""comments_count"":2,""cr..."
3,"{""patterns"": {""149679"":{""comments_count"":0,""cr..."
4,"{""patterns"": {""18608"":{""comments_count"":0,""cre..."


In [12]:
df.columns = ['patterns']

In [13]:
df['pattern_id'] = df['patterns'].str.extract('{"patterns": {"([0-9]+)') # extract pattern IDs from patterns column
df.set_index('pattern_id');

In [14]:
# extract pattern names from patterns column
df['pattern_name'] = df['patterns'].str.extract('"id":[0-9]+,"name":"([^\s"]+(?:\s+[^\s"]+){,9})","pdf_url"', expand=False)

# extract URL for "medium2"-size image from patterns column
df['medium2_urls'] = df['patterns'].str.extract('"medium2_url":"(https:\/\/\S+_medium2.(?:jpg|jpeg|gif|png))"', re.IGNORECASE, expand=False)

# extract author name from patterns column
df['author_name'] = df['patterns'].str.extract('"knitting_pattern_count":[0-9]+,"name":"([^\s"]+(?:\s+[^\s"]+){,9})"', expand=False)

In [15]:
df.head(5)

,patterns,pattern_id,pattern_name,medium2_urls,author_name
0,"{""patterns"": {""18602"":{""comments_count"":0,""cre...",18602,Slice of Pi Tam/Beret,https://images4-e.ravelrycache.com/uploads/sno...,Sarah Bradberry
1,"{""patterns"": {""280747"":{""comments_count"":0,""cr...",280747,Urban Hunter,https://images4-b.ravelrycache.com/uploads/Woo...,Cheryl Andrews
2,"{""patterns"": {""149677"":{""comments_count"":2,""cr...",149677,Festive Winter Bonnet,NaN,Concetta Phillipps
3,"{""patterns"": {""149679"":{""comments_count"":0,""cr...",149679,Fancy Rib Hat,NaN,Sandy Scoville
4,"{""patterns"": {""18608"":{""comments_count"":0,""cre...",18608,Pixie Bonnet,https://images4-e.ravelrycache.com/uploads/sno...,Sarah Bradberry


In [17]:
csv_filename = '/Users/ssivek/Insight_Projects/Ravelry_Project/Data/patterns' + str(last) + '.csv' 
export_csv = df.to_csv (csv_filename, index = None, header=True) 